---
# Re-evaluation of DDX56 expression in GNS lines

L.Richards  
October 2019  
  
---

---
### 1.0 Plot additional genes for co-expression with DDX56
---

https://docs.google.com/spreadsheets/d/1UxN5XvyCmyVXOSmSkb6WU859ClF1q0d5vYOYD4d9jtY/edit#gid=0

These genes **are not** in the data:  
 [1] "TNFSF10" "P14ARF"  "IFN"     "IRF4"    "CHAT"    "H1FOO"   "IIAE7"  
 [8] "ISG12A"  "ISG12"   "TRAIL"  
 
 These genes **are** in the data:  
  [1] "IRF3"   "IFI27"  "IFIT1"  "RPL18A" "TP53"   "RPL18"  "AK1"    "RPL10A"  
 [9] "RPL31"  "RPL36A" "RPL37"  "RPL39"  "RPL5"   "SRSF10" "RPL37A" "SRSF10"  
[17] "CBFB"   "TRMT6"  "PES1"   "WDHD1"  "LYAR"   "CBX3"  

I would be weary of the RPL genes, their expression tends to modulate due to technical reasons.

---
### 2.0 Subtype cells in G523 for TCGA subtypes
---



In [ ]:
##isolate random scores
                                  
random.scores <- scored@meta.data[grep("Random", colnames(scored@meta.data))]
scored@meta.data <- scored@meta.data[,-grep("Random", colnames(scored@meta.data))]
                                  

### Define top 5% cutoff for classification and plot

        
        cutoffs <- c()
            
        for (sig.name in names(subtypes)){
         
            print(sig.name)
            range <- grep(sig.name, colnames(random.scores))
            #print(range)
            
            bb <- as.vector(as.matrix(random.scores[,range]))
            print(length(bb))
            
            d <- density(bb)
            cutoff <- qnorm(0.95, mean = mean(bb), sd = sd(bb))
            print(cutoff)
            
            filename <- paste0(sig.name, "AddModuleScore_RandomListCutoff.pdf")
            pdf(filename)

            plot_raw <- plot(d,
                 main = paste0(sig.name," - Random Signatures"),
                 xlab = "AddModuleScore()"
                )
            legend("topright",
            legend = c(paste("Mean:", round(mean(bb),3)),
                          paste("Sd:", round(sd(bb), 3)),
                          paste("Cutoff:", round(cutoff,3))
                     ),
               bty = 'n',
               cex = 1
          )
        
        abline(v=cutoff, lty = 2, col = "red")
        dev.off()
        cutoffs <- c(cutoffs, cutoff)
        
        }
        
        names(cutoffs) <- names(subtypes)
        cutoffs

In [ ]:
#### classify the cells

meta <- scored@meta.data
            
for (i in 1:length(subtypes)){
    
    subset <- meta[ ,grep(names(subtypes[i]), colnames(meta))]
    
    col.name <- paste0("Classify_", names(subtypes[i]), "_", 
                           round(as.numeric(cutoffs[grep(names(subtypes[i]), names(cutoffs))]), 4)
                           )
    
    meta[col.name] <- subset > as.numeric(cutoffs[grep(names(subtypes[i]), names(cutoffs))])
        
    }


### make a single classification co

meta$Class <- do.call(paste, c(meta[17:20], sep="-"))
            
meta$Class <- gsub("FALSE-FALSE-FALSE-FALSE", "Unclassified", meta$Class)
meta$Class <- gsub("FALSE-FALSE-FALSE-TRUE", "CLA", meta$Class)
meta$Class <- gsub("FALSE-FALSE-TRUE-FALSE", "MES", meta$Class)
meta$Class <- gsub("FALSE-FALSE-TRUE-TRUE", "MES+CLA", meta$Class)
meta$Class <- gsub("FALSE-TRUE-FALSE-FALSE", "PRO", meta$Class)
meta$Class <- gsub("FALSE-TRUE-FALSE-TRUE", "PRO+CLA", meta$Class)

meta$Class <- gsub("FALSE-TRUE-TRUE-FALSE", "PRO+MES", meta$Class)
meta$Class <- gsub("FALSE-TRUE-TRUE-TRUE", "PRO+MES+CLA", meta$Class)
meta$Class <- gsub("TRUE-FALSE-FALSE-FALSE", "NEU", meta$Class)
meta$Class <- gsub("TRUE-FALSE-FALSE-TRUE", "NEU+CLA", meta$Class)
meta$Class <- gsub("TRUE-FALSE-TRUE-FALSE", "NEU+MES", meta$Class)
meta$Class <- gsub("TRUE-FALSE-TRUE-TRUE", "NEU+MES+CLA", meta$Class)

meta$Class <- gsub("TRUE-TRUE-FALSE-FALSE", "NEU+PRO", meta$Class)
meta$Class <- gsub("TRUE-TRUE-FALSE-TRUE", "NEU+PRO+CLA", meta$Class)
meta$Class <- gsub("TRUE-TRUE-TRUE-FALSE", "NEU+PRO+MES", meta$Class)
meta$Class <- gsub("TRUE-TRUE-TRUE-TRUE", "NEU+PRO+MES+CLA", meta$Class)

meta <- meta[ ,c("DDX56_class", 
                 "Neural1", 
                 "Proneural2",
                 "Mesenchymal3",
                 "Classical4",
                 "Classify_Neural_0.05",
                 "Classify_Proneural_0.047",
                 "Classify_Mesenchymal_0.0448",
                 "Classify_Classical_0.0446",
                 "Class")]

In [ ]:
### save the data

write.csv(meta, file = paste0("DDX56_ScoredTCGA", ".csv"), row.names = T)
            
        save(random.sigs,
             subtypes,
             scored,
             random.scores,
             meta,
             file = "DDX56_ScoredTCGA.Rdata"
            )